In [5]:
import warnings
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')


In [6]:
data = pd.read_csv('data/data.csv', encoding='utf-8-sig') 
data.head()


,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME(UTC),ORIGIN TIME(IST),MAGNITUDE(Mw1),MAGNITUDE(Mw2),MAGNITUDE(Mb1),MAGNITUDE(Mb2),MAGNITUDE(Ms),MAGNITUDE(Ml),LAT (N),LONG (E),DEPTH (km),Intensity(MM),Intensity(MMI),Intensity(MME),LOCATION,REFERENCE
0,1,-2474,0.0,0.0,NaN,NaN,7.5000,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
1,2,-325,0.0,0.0,NaN,NaN,7.5000,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0.0,NaN,NaN,NaN,NaN,Dr STGR
2,3,25,0.0,0.0,NaN,NaN,7.5000,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0.0,NaN,NaN,NaN,NaN,Dr STGR
3,4,26,5.0,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC
4,5,26,5.0,10.0,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,6.075520196,5.999685205,26,97,80.0,NaN,NaN,NaN,Myanmar,G-R


In [16]:
type(data['LAT (N)'][23])

float

In [47]:
l = []
l.append(["ORIGIN TIME(IST)",data['ORIGIN TIME(IST)'].isnull().sum(), data['ORIGIN TIME(IST)'].notnull().sum()])
l.append(["ORIGIN TIME(UTC)",data['ORIGIN TIME(UTC)'].isnull().sum(), data['ORIGIN TIME(UTC)'].notnull().sum()])
l.append(["Intensity(MMI)",data['Intensity(MMI)'].isnull().sum(), data['Intensity(MMI)'].notnull().sum()])
l.append(["Intensity(MME)", data['Intensity(MME)'].isnull().sum(), data['Intensity(MME)'].notnull().sum()])
l.append(["Intensity(MM)",data['Intensity(MM)'].isnull().sum(), data['Intensity(MM)'].notnull().sum()])
l.append(["LOCATION",data['LOCATION'].isnull().sum(), data['LOCATION'].notnull().sum()])
print(tabulate(l, headers=['feature','Nulls','Not Nulls'], tablefmt='orgtbl'))

| feature          |   Nulls |   Not Nulls |
|------------------+---------+-------------|
| ORIGIN TIME(IST) |   52563 |         426 |
| ORIGIN TIME(UTC) |   31803 |       21186 |
| Intensity(MMI)   |   52989 |           0 |
| Intensity(MME)   |   52989 |           0 |
| Intensity(MM)    |   52948 |          41 |
| LOCATION         |   43615 |        9374 |


We can see that the locations, origin times and intensities are mostly null. So, we cannot even reconstruct or interpolate them based on other values. So dropping those columns. Also, source is not useful in classification and so is the date. Hence dropping it too. We need to predict only Mw.(Note - Mw1 and Mw2 differ only in precision)


In [17]:
features = ['LAT (N)', 'LONG (E)', 'DEPTH (km)']
predict_features = ['MAGNITUDE(Mw1)']
X = data[features]
y = data[predict_features]
X.head()

,LAT (N),LONG (E),DEPTH (km)
0,71,24,0.0
1,71,24,0.0
2,72.9,33.72,0.0
3,17.3,80.1,NaN
4,26,97,80.0


In [49]:
y.head()

,MAGNITUDE(Mw1)
0,7.5000
1,7.5000
2,7.5000
3,6.1397
4,6.1397


Not all Latitude and Longitude values are numbers. From 50573, they are strings. No necessary pattern in source for what all are given as strings. All the values given as string have source as IMD. But not all IMD given values are strings. 

In [14]:
# def clean_valuesN(x):
#     """ If the value is a string, then remove currency symbol and delimiters
#     otherwise, the value is numeric and can be converted
#     """
#     if isinstance(x, str):
#         return(x.replace(' ','').replace('ï¿½', '').replace('\?','').replace('N',''))
#     return(x)

# def clean_valuesS(x):
#     """ If the value is a string, then remove currency symbol and delimiters
#     otherwise, the value is numeric and can be converted
#     """
#     if isinstance(x, str):
#         return(x.replace(' ','').replace('ï¿½', '').replace('\?','').replace('S',''))
#     return(x)

# def clean_valuesE(x):
#     """ If the value is a string, then remove currency symbol and delimiters
#     otherwise, the value is numeric and can be converted
#     """
#     if isinstance(x, str):
#         return(x.replace(' ','').replace('ï¿½', '').replace('\?','').replace('E',''))
#     return(x)

# def clean_valuesW(x):
#     """ If the value is a string, then remove currency symbol and delimiters
#     otherwise, the value is numeric and can be converted
#     """
#     if isinstance(x, str):
#         return(x.replace(' ','').replace('ï¿½', '').replace('\?','').replace('W',''))
#     return(x)
import re
def clean_values(x):
    """
    Take only numbers in the string
    """
    regex = r'-?\d+.?\d?'
    x = str(x)
    ret = re.findall(regex, x)
    return ret[0]
    

In [22]:
# Handling string values in Latitudes and Longitudes.
# X['LAT (N)'] = X['LAT (N)'].apply(clean_valuesN).apply(clean_valuesS)
# X['LONG (E)'] = X['LONG (E)'].apply(clean_valuesE).apply(clean_valuesW)
X['LAT (N)'] = X['LAT (N)'].apply(clean_values)
X['LONG (E)'] = X['LONG (E)'].apply(clean_values)
# X['LAT (N)'] = X['LAT (N)'].astype('float')
# X['LONG (E)'] = X['LONG (E)'].astype('float')

print(X['LAT (N)'].tail())
print(X['LONG (E)'].tail())
# convert to csv and check to see what is the isse
# newdt = X['LAT (N)']
# newdt.to_csv('lat.csv')

ValueError: could not convert string to float: '6,2'

Setting a random threshold of 4.5 currently and creating the prediction class. Need to do this smartly.


In [52]:
thresh  = 4.5
y['MAGNITUDE(Mw1)'].values[y['MAGNITUDE(Mw1)'] < thresh] = 0
y['MAGNITUDE(Mw1)'].values[y['MAGNITUDE(Mw1)'] >= thresh ] = 1
y.head()

,MAGNITUDE(Mw1)
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [53]:
print("Total number of samples: ",y.size)
print("Number of negative samples: ",y['MAGNITUDE(Mw1)'].value_counts()[0.0])
print("Number of positive samples: ",y['MAGNITUDE(Mw1)'].value_counts()[1.0])
print("Number of null values: ", y['MAGNITUDE(Mw1)'].isnull().sum())

Total number of samples:  52989
Number of negative samples:  21736
Number of positive samples:  19199
Number of null values:  12054


Dropping out all samples with nan in magnitude column

In [54]:
X = X[pd.notnull(y['MAGNITUDE(Mw1)'])]
y = y[pd.notnull(y['MAGNITUDE(Mw1)'])]
print("Total number of samples: ",y.size)
print("Number of negative samples: ",y['MAGNITUDE(Mw1)'].value_counts()[0.0])
print("Number of positive samples: ",y['MAGNITUDE(Mw1)'].value_counts()[1.0])
print("Number of null values: ", y['MAGNITUDE(Mw1)'].isnull().sum())

Total number of samples:  40935
Number of negative samples:  21736
Number of positive samples:  19199
Number of null values:  0


Since the number of positive samples and negative samples are nearly equal,we need not worry about class imbalance.

In [55]:
# HANDLE MISSING VALUES IN DEPTH COLUMN
# https://www.kaggle.com/learn/data-cleaning
# Filling with the average value for now
mean_depth = X['DEPTH (km)'].mean()
X['DEPTH (km)'] = X['DEPTH (km)'].fillna(mean_depth)
X.head()

,LAT (N),LONG (E),DEPTH (km)
0,71,24,0.000000
1,71,24,0.000000
2,72.9,33.72,0.000000
3,17.3,80.1,47.063533
4,26,97,80.000000


In [56]:
# Splitting the dataset into training, testing and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [57]:
print("Number of positive samples in train set",y_train['MAGNITUDE(Mw1)'].value_counts()[1.0])
print("Number of negative samples in train set",y_train['MAGNITUDE(Mw1)'].value_counts()[0.0])
print("Number of positive samples in val set",y_val['MAGNITUDE(Mw1)'].value_counts()[1.0])
print("Number of negative samples in val set",y_val['MAGNITUDE(Mw1)'].value_counts()[0.0])
print("Number of positive samples in test set",y_test['MAGNITUDE(Mw1)'].value_counts()[1.0])
print("Number of negative samples in test set",y_test['MAGNITUDE(Mw1)'].value_counts()[0.0])
print("We can see that all the sets are fairly balanced")

Number of positive samples in train set 12176
Number of negative samples in train set 14022
Number of positive samples in val set 3100
Number of negative samples in val set 3450
Number of positive samples in test set 3923
Number of negative samples in test set 4264
We can see that all the sets are fairly balanced


### KNN


In [34]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train,y_train)
# accuracy, roc curve and other statistics